# MERFISH whole brain spatial transcriptomics (part 1)

The MERFISH data consists of 59 coronal sections (10 micron thick) covering the whole anterior to posterior extent of the brain (200 micron apart) from a single adult male mouse. Each section is assayed using the MERSCOPE platform with a 500 gene panel designed to enable us to map cells to the whole mouse brain taxonomy.

The MERSCOPE platform uses combinatorial labeling, sequential imaging and error robust barcodes to obtain quantitative (counts) RNA-expression of 100s-1000s of genes. Each probe binds to a specific transcripts resulting in “dots” in the image. Cells are segmented using CellPose software based on DAPI/PolyT stains.

Gene expression data for each section is stored as individual h5ad files. The puprose of this set of notebooks is to provide and overview of the data, the file organization and how to combine the data and metadata through example use cases.

In [1]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import json
import matplotlib.pyplot as plt
import requests

In [2]:
user_home = os.getenv('HOME')
download_base = os.path.join(user_home,'abc_download_root')

url = 'https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/releases/20230630/manifest.json'
manifest = json.loads(requests.get(url).text)
    
metadata = manifest['file_listing']['MERFISH-C57BL6J-638850']['metadata']

In [3]:
view_directory = os.path.join( download_base, 
                               manifest['directory_listing']['MERFISH-C57BL6J-638850']['directories']['metadata']['relative_path'], 
                              'views')
cache_views = False
if cache_views :
    os.makedirs( view_directory, exist_ok=True )

## Data overview

### Cell metadata

Essential cell metadata is stored as a dataframe. Each row represents one cell indexed by a *cell label*.

Each cell is associated with a *brain section label*, *donor label*, *donor genotype*, *donor sex* and *matrix_prefix* identifying which data package this cell is part of. Each cell also has a set of x, y, z coordinates generated by rotating each section so that it is upright with midline approximatedly in the middle of the frame. The z coordinate takes into account the phyiscal spacing between each section.

Each cell is mapped to the whole mouse brain using a hierachical correlation method, resulting in the assignment of a *cluster alias" and *average correlation score*. Score of less than 0.5 is considered a poor mapping result and should not be used in downstream analysis.

In [4]:
rpath = metadata['cell_metadata']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cell = pd.read_csv(file, dtype={"cell_label":str})
cell.set_index('cell_label',inplace=True)
print("Number of cells = ", len(cell))
cell.head(5)

Number of cells =  4330907


,brain_section_label,cluster_alias,average_correlation_score,matrix_label,donor_label,low_quality_mapping,donor_genotype,donor_sex,x,y,z
cell_label,,,,,,,,,,,
1015221640100570419,C57BL6J-638850.01,4090,0.582276,C57BL6J-638850.01,C57BL6J-638850,False,wt/wt,M,5.171280,6.489884,0.0
1015221640100590598,C57BL6J-638850.01,4090,0.410491,C57BL6J-638850.01,C57BL6J-638850,True,wt/wt,M,NaN,NaN,NaN
1015221640100820600,C57BL6J-638850.01,4090,0.568130,C57BL6J-638850.01,C57BL6J-638850,False,wt/wt,M,6.547826,5.712030,0.0
1015221640100580476,C57BL6J-638850.01,4090,0.417244,C57BL6J-638850.01,C57BL6J-638850,True,wt/wt,M,NaN,NaN,NaN
1015221640100580189,C57BL6J-638850.01,4090,0.453351,C57BL6J-638850.01,C57BL6J-638850,True,wt/wt,M,NaN,NaN,NaN


We define a small helper fuction that uses pandas groupby function to see how many unique items are associated for each field and list them out if the number of items is small.

In [5]:
def print_column_info(df) :
    
    for c in df.columns :
        grouped = df[[c]].groupby(c).count()
        members = ''
        if len(grouped) < 30 :
            members = str(list(grouped.index))
        print("Number of unique %s = %d %s" % (c,len(grouped),members))

In [6]:
print_column_info(cell)

Number of unique brain_section_label = 59 
Number of unique cluster_alias = 5194 
Number of unique average_correlation_score = 4328111 
Number of unique matrix_label = 59 
Number of unique donor_label = 1 ['C57BL6J-638850']
Number of unique low_quality_mapping = 2 [False, True]
Number of unique donor_genotype = 1 ['wt/wt']
Number of unique donor_sex = 1 ['M']
Number of unique x = 3940980 
Number of unique y = 3940980 
Number of unique z = 59 


We can also bring in the pivot table from the "cluster annotation tutorial" to associate each cell with terms at each cell type classification level and the corresponding color.

In [7]:
taxonomy_metadata = manifest['file_listing']['WMB-taxonomy']['metadata']

In [8]:
rpath = taxonomy_metadata['cluster_to_cluster_annotation_membership_pivoted']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cluster_details = pd.read_csv(file)
cluster_details.set_index('cluster_alias', inplace=True)
cluster_details.head(5)

,neurotransmitter,division,class,subclass,supertype,cluster
cluster_alias,,,,,,
1,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3
2,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0328 L2 IT PPP-APr Glut_3
3,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0080 L2 IT PPP-APr Glut_2,0323 L2 IT PPP-APr Glut_2
4,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0080 L2 IT PPP-APr Glut_2,0324 L2 IT PPP-APr Glut_2
5,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0080 L2 IT PPP-APr Glut_2,0326 L2 IT PPP-APr Glut_2


In [9]:
rpath = taxonomy_metadata['cluster_to_cluster_annotation_membership_color']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cluster_colors = pd.read_csv(file)
cluster_colors.set_index('cluster_alias', inplace=True)
cluster_colors.head(5)

,neurotransmitter_color,division_color,class_color,subclass_color,supertype_color,cluster_color
cluster_alias,,,,,,
1,#2B93DF,#4D58FF,#f74294,#0F6632,#266DFF,#26FF8C
2,#2B93DF,#4D58FF,#f74294,#0F6632,#266DFF,#280099
3,#2B93DF,#4D58FF,#f74294,#0F6632,#002BCC,#5CCCA0
4,#2B93DF,#4D58FF,#f74294,#0F6632,#002BCC,#CC6D1F
5,#2B93DF,#4D58FF,#f74294,#0F6632,#002BCC,#FF8E26


In [10]:
cell_extended = cell.join(cluster_details,on='cluster_alias')
cell_extended = cell_extended.join(cluster_colors,on='cluster_alias')
cell_extended.head(5)

,brain_section_label,cluster_alias,average_correlation_score,matrix_label,donor_label,low_quality_mapping,donor_genotype,donor_sex,x,y,...,class,subclass,supertype,cluster,neurotransmitter_color,division_color,class_color,subclass_color,supertype_color,cluster_color
cell_label,,,,,,,,,,,,,,,,,,,,,
1015221640100570419,C57BL6J-638850.01,4090,0.582276,C57BL6J-638850.01,C57BL6J-638850,False,wt/wt,M,5.171280,6.489884,...,22 MY GABA,282 MY Calcb Chol,0999 MY Calcb Chol,5100 MY Calcb Chol,#73E785,#9F3DCC,#E5C865,#FF4D81,#2E5499,#91FF4D
1015221640100590598,C57BL6J-638850.01,4090,0.410491,C57BL6J-638850.01,C57BL6J-638850,True,wt/wt,M,NaN,NaN,...,22 MY GABA,282 MY Calcb Chol,0999 MY Calcb Chol,5100 MY Calcb Chol,#73E785,#9F3DCC,#E5C865,#FF4D81,#2E5499,#91FF4D
1015221640100820600,C57BL6J-638850.01,4090,0.568130,C57BL6J-638850.01,C57BL6J-638850,False,wt/wt,M,6.547826,5.712030,...,22 MY GABA,282 MY Calcb Chol,0999 MY Calcb Chol,5100 MY Calcb Chol,#73E785,#9F3DCC,#E5C865,#FF4D81,#2E5499,#91FF4D
1015221640100580476,C57BL6J-638850.01,4090,0.417244,C57BL6J-638850.01,C57BL6J-638850,True,wt/wt,M,NaN,NaN,...,22 MY GABA,282 MY Calcb Chol,0999 MY Calcb Chol,5100 MY Calcb Chol,#73E785,#9F3DCC,#E5C865,#FF4D81,#2E5499,#91FF4D
1015221640100580189,C57BL6J-638850.01,4090,0.453351,C57BL6J-638850.01,C57BL6J-638850,True,wt/wt,M,NaN,NaN,...,22 MY GABA,282 MY Calcb Chol,0999 MY Calcb Chol,5100 MY Calcb Chol,#73E785,#9F3DCC,#E5C865,#FF4D81,#2E5499,#91FF4D


In [ ]:
print_column_info(cell_extended)

Number of unique brain_section_label = 59 
Number of unique cluster_alias = 5194 
Number of unique average_correlation_score = 4328111 
Number of unique matrix_label = 59 
Number of unique donor_label = 1 ['C57BL6J-638850']
Number of unique low_quality_mapping = 2 [False, True]
Number of unique donor_genotype = 1 ['wt/wt']
Number of unique donor_sex = 1 ['M']
Number of unique x = 3940980 
Number of unique y = 3940980 
Number of unique z = 59 
Number of unique neurotransmitter = 9 ['Chol', 'Dopa', 'GABA', 'GABA-Glyc', 'Glut', 'Glut-GABA', 'Hist', 'Nora', 'Sero']
Number of unique division = 7 ['1 Pallium glutamatergic', '2 Subpallium GABAergic', '3 PAL-sAMY-TH-HY-MB-HB neuronal', '4 CBX-MOB-other neuronal', '5 Neuroglial', '6 Vascular', '7 Immune']
Number of unique class = 32 
Number of unique subclass = 307 
Number of unique supertype = 1114 
Number of unique cluster = 5194 


For convenience, we can cache this view for later reuse.

In [ ]:
if cache_views :
    file = os.path.join( view_directory, 'cell_metadata_with_cluster_annotation.csv')
    cell_extended.to_csv( file )

### Gene expression matrices

The expression of each section is stored as an individual anndata h5ad file with minimal metadata. In this next section, we provide example code on how to open the file and connect with the rich cell level metadata discussed above.

For each section, there are two h5ad files one storing the raw counts and the other log normalization of it.

In [ ]:
expression_matrices = manifest['file_listing']['MERFISH-C57BL6J-638850']['expression_matrices']

In [ ]:
expression_matrices['C57BL6J-638850.38']

In [ ]:
rpath = expression_matrices['C57BL6J-638850.38']['log2']['files']['h5ad']['relative_path']
file = os.path.join( download_base, rpath)
print(file)

We use the anndata's *read_h5ad* function to open the package for the log2 normalization file. The "backed='r'" makes use of the lazy loading functionality to only load required data. By default, anndata will load the entire expression matrix in memory.

In [ ]:
adata = anndata.read_h5ad(file,backed='r')
print(adata)

Genes are represented as "variables". For this data, the var dataframe is indexed by the Ensembl gene identifier with one metadata column gene symbol.

In [ ]:
print("Number of genes = ", len(adata.var))
adata.var.index[0:5]

Cells are represented as "observations". We can easily connect the cells in each section to the extended cell metadata and filter out poorly mapped cells with simple pandas functions.

In [ ]:
cell_filtered = cell_extended.loc[adata.obs.index]
pred = cell_filtered['low_quality_mapping'] == False
cell_filtered = cell_filtered[pred]

print("Number of cells = ", len(cell_filtered))
print_column_info( cell_filtered )

We define a small helper function *plot section* to visualize the cells in anatomical context colorized by: neurotransmitter identity, cell types division, class and subclass.

In [ ]:
def plot_section( xx, yy, cc=None, val=None, fig_width = 8, fig_height = 8, cmap=None ) :
    fig, ax = plt.subplots()
    fig.set_size_inches(fig_width, fig_height)
    if cmap is not None :
        plt.scatter(xx,yy,s=0.5,c=val,marker='.',cmap=cmap)
    elif cc is not None :
        plt.scatter(xx,yy,s=0.5,color=cc,marker='.')
    ax.set_ylim(11,0)
    ax.set_xlim(0,11)
    ax.axis('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    return fig, ax

In [ ]:
fig, ax = plot_section(cell_filtered['x'], cell_filtered['y'],cell_filtered['neurotransmitter_color'])
res = ax.set_title("Neuortransmitter Identity")

In [ ]:
fig, ax = plot_section(cell_filtered['x'], cell_filtered['y'],cell_filtered['division_color'])
res = ax.set_title("Cell Types: Division")

In [ ]:
fig, ax = plot_section(cell_filtered['x'], cell_filtered['y'],cell_filtered['class_color'])
res = ax.set_title("Cell Types: Class")

In [ ]:
fig, ax = plot_section(cell_filtered['x'], cell_filtered['y'],cell_filtered['subclass_color'])
res = ax.set_title("Cell Types: Subclass")

## Example use cases

In this section, we explore the same two use cases as with the 10x data. The first example looks at the expression of nine canonical neurotransmitter transporter genes and the second the expression of gene Tac2. To support these use cases, we will create a smaller submatrix (all cells and 10 genes) that read it into memory. *Note this operation takes around 10 seconds*.

In [ ]:
ntgenes = ['Slc17a7','Slc17a6','Slc17a8','Slc32a1','Slc6a5','Slc6a3','Slc6a4']
exgenes = ['Tac2']
gnames = ntgenes + exgenes
pred = [x in gnames for x in adata.var.gene_symbol]
gene_filtered = adata.var[pred]
gene_filtered

In [ ]:
start = time.process_time()
asubset = adata[cell_filtered.index,gene_filtered.index].to_memory()
print("time taken: ", time.process_time() - start)
print(asubset)

We define two helper functions to (1) *create_expression_dataframe*: create joined gene expression and cell metadata dataframe based for a set of input genes and (2) *aggregate_by_metadata* which computes the average expression for each term in a given category.

In [ ]:
def create_expression_dataframe( ad, gf ) :
    gdata = ad[:,gf.index].to_df()
    gdata.columns = gf.gene_symbol
    joined = cell_filtered.join( gdata )
    return joined

In [ ]:
def aggregate_by_metadata( df, gnames, value, sort = False ) :
    grouped = df.groupby(value)[gnames].mean()
    if sort :
        grouped = grouped.sort_values(by=gnames[0],ascending=False)
    return grouped

### Expression of canonical neurotransmitter transporter genes

During analysis, clusters were assigned neurotransmitter identities based on the expression of of canonical neurotransmitter transporter genes. In this example, we create a dataframe comprising of cells in the 'C57BL6J-638850.40' section and expression of the 9 solute carrier family genes. We then group the cells by the assigned neurotransmitter class and compute the mean expression for each group and visualized as a colorized table. The results are similar of the 10x data restricted to cells from the thalamus.

In [ ]:
pred = [x in ntgenes for x in asubset.var.gene_symbol]
gf = asubset.var[pred]

ntexp = create_expression_dataframe( asubset, gf )
agg = aggregate_by_metadata(ntexp, gf.gene_symbol, 'neurotransmitter')
agg = agg[ntgenes]
agg.style.background_gradient(cmap='Reds')

The expression data can also be visualized as scatter point in anatomical context. Gene Slc17a7 is highly enriched in the cortex , hippocampus, medial habenula with some expression in the thalamus. On the other hand gene Slc32a1 is enriched in the striatum with sparse expression on the cortical plate.

In [ ]:
fig, ax = plot_section(ntexp['x'], ntexp['y'],val=ntexp['Slc17a7'],cmap='Reds')

In [ ]:
fig, ax = plot_section(ntexp['x'], ntexp['y'],val=ntexp['Slc32a1'],cmap='Reds')

### Expression of Tachykinin 2 (Tac2) in the whole brain

In mice, the tachykinin 2 (Tac2) gene encodes neuropeptide called neurokinin B (NkB). Tac2 is produced by neurons in specific regions of the brain know to be invovled in emotion and social behavior. Based on [ISH data](https://mouse.brain-map.org/experiment/siv?id=77279001&imageId=77284584&initImage=ish&coordSystem=pixel&x=5384.5&y=3832.5&z=3) from the Allen Mouse Brain Atlas, Tac 2 is sparsely expressed in the mouse isocortex and densely enriched is specific subcortical regions such the medial habenula (MH), the amygdala and hypothalamus.

<br>
<div>
<img src="Tac2_ISH_Coronal_Section.JPG" width="600"/>
</div>


In this example, we create a dataframe comprising of cells in section 'C57BL6J-638850.40' and expression values of Tac2 in those cells.

In [ ]:
gf = asubset.var[asubset.var.gene_symbol == 'Tac2']
tac2 = create_expression_dataframe( asubset, gf )

Grouping cells by neurotransmitter identites and computing the mean expression in each group, we can observed that Tac2 gene is highly enriched in cholinergic cell types with some expression in Glut-GABA types.

In [ ]:
agg = aggregate_by_metadata(tac2, gf.gene_symbol, 'neurotransmitter', True).head(8)
agg.style.background_gradient(cmap='Reds')

Grouping cells by cell types class, shows that Tac2 is enriched in class "08 MH-LH Glut" with cells restricted to the medial (MH) and lateral (LH) habenula and a mixture of glutamatergic and cholinergic types.

In [ ]:
agg = aggregate_by_metadata(tac2, gf.gene_symbol, 'class', True).head(8)
agg.style.background_gradient(cmap='Reds')

At the next level, grouping by subclass, shows Tac2 is enriched in sublcasses with specificity to the medial habenula (MH), bed muclei of the stria terminalis (BST) and main olfactory blub (MOB).

In [ ]:
agg = aggregate_by_metadata(tac2, gf.gene_symbol, 'subclass', True).head(8)
agg.style.background_gradient(cmap='Reds')

Plotting Tac2 expression in anatomical coordinates results in a image that is comparable to the ISH image data showing enriched expression in the medial habenula, amydala and hypothalamus and sparse expression in the cortex.

In [ ]:
fig,ax = plot_section(tac2['x'], tac2['y'],val=tac2['Tac2'],cmap='Reds')
res = ax.set_title('Gene Tac2')

Close h5ad file and clean up

In [ ]:
adata.file.close()
del adata